In [1]:
import seaborn as sns
import random as rand
import matplotlib.pyplot as plt
# from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import RobustScaler, PowerTransformer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, auc
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('Insurance_Cross_Selling/train_transformed.csv')
train.head()

,Gender,Age,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response,special_channel_152.0,special_channel_26.0,special_channels,is_young_driver,is_old_driver,is_special_region
0,1.0,21,35.0,0,0.0,1.0,2.347749,124.0,187,0,0,0,0,1,0,0
1,1.0,43,28.0,0,2.0,1.0,1.911034,26.0,288,1,0,1,1,0,0,1
2,0.0,25,14.0,1,1.0,0.0,0.438761,152.0,254,0,1,0,1,0,0,0
3,0.0,35,1.0,0,0.0,1.0,-2.059687,156.0,76,0,0,0,0,0,0,0
4,0.0,36,15.0,1,0.0,0.0,0.008960,152.0,294,0,1,0,1,0,0,0


In [3]:
test = pd.read_csv('Insurance_Cross_Selling/test_transformed.csv')
test.head()

,Gender,Age,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,special_channel_152.0,special_channel_26.0,special_channels,is_young_driver,is_old_driver,is_special_region
0,0.0,20,47.0,0,1.0,0.0,-2.059687,160.0,228,0,0,0,1,0,0
1,1.0,47,28.0,0,0.0,1.0,0.399252,124.0,123,0,0,0,0,0,1
2,1.0,47,43.0,0,0.0,1.0,-2.059687,26.0,271,0,1,1,0,0,0
3,0.0,22,47.0,1,1.0,0.0,-0.516580,152.0,115,1,0,1,1,0,0
4,1.0,51,19.0,0,0.0,0.0,0.161634,124.0,148,0,0,0,0,0,0


In [4]:
print(train.shape, test.shape)

(11504798, 16) (7669866, 15)


In [5]:
X, y = train.drop('Response', axis=1), train['Response']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [7]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(8053358, 15) (3451440, 15) (8053358,) (3451440,)


#### Logistic Regression

In [9]:
lr = LogisticRegression()

lr.fit(X_train, y_train)
y_pred_lgr = lr.predict_proba(X_test)[:,1]

In [ ]:
# y_pred1 = y_pred[:,1]
# print(y_pred.shape, y_pred1.shape)
# print(y_pred[:5])
# print(y_pred1[:5])

In [10]:
score = roc_auc_score(y_test, y_pred_lgr)
print(f"Score : {score}")

Score : 0.8449137147267332


In [12]:
# !pip3 install xgboost

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
/usr/share/python-wheels/urllib3-1.25.8-py2.py3-none-any.whl/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pypi.ngc.nvidia.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
     |████████████████████████████████| 4.5 MB 1.3 MB/s eta 0:00:01


In [13]:
from xgboost import XGBClassifier

In [14]:
xgb = XGBClassifier()

xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict_proba(X_test)[:,1]

In [15]:
score = roc_auc_score(y_test, y_pred_xgb)
print(f"Score : {score}")

Score : 0.8780805506230185


In [16]:
xgb_tuned = XGBClassifier(objective='binary:logistic',
    eval_metric='auc',
    device='cuda',
    n_estimators=1500,
    learning_rate=0.1,
    max_depth=10,
    min_child_weight=25,
    subsample=0.9,
    colsample_bynode=0.9,
    reg_lambda=20,
    tree_method='approx',
    max_bin=1024
    )

In [17]:
xgb_tuned.fit(X_train, y_train)
y_pred_xgb_tuned = xgb_tuned.predict_proba(X_test)[:,1]

In [18]:
score = roc_auc_score(y_test, y_pred_xgb_tuned)
print(f'Score: {score}')

Score: 0.8840432162608002


In [19]:
# test_predictions = xgb.predict_proba(test)
test_predictions = xgb_tuned.predict_proba(test)

In [20]:
test_1 = pd.read_csv('Insurance_Cross_Selling/test.csv')

In [21]:
test_1.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,11504798,Female,20,1,47.0,0,< 1 Year,No,2630.0,160.0,228
1,11504799,Male,47,1,28.0,0,1-2 Year,Yes,37483.0,124.0,123
2,11504800,Male,47,1,43.0,0,1-2 Year,Yes,2630.0,26.0,271
3,11504801,Female,22,1,47.0,1,< 1 Year,No,24502.0,152.0,115
4,11504802,Male,51,1,19.0,0,1-2 Year,No,34115.0,124.0,148


In [22]:
res_df = pd.DataFrame({
    'id':test_1['id'],
    'Response':test_predictions[:,1]
})

In [23]:
res_df.to_csv('Insurance_Cross_Selling/xgb_tuned_submission.csv', index=False)